In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
import tensorflow as tf
import tensorboard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# encoding error with degrees symbol in utf-8
df = pd.read_csv("./SeoulBikeData.csv",
                 encoding_errors="ignore")
rename_dict = {}
rename_dict["Temperature(C)"] = "Temperature"
rename_dict["Humidity(%)"] = "Humidity"
rename_dict["Wind speed (m/s)"] = "Wind speed"
rename_dict["Visibility (10m)"] = "Visibility"
rename_dict["Dew point temperature(C)"] = "Dew point temperature"
rename_dict["Solar Radiation (MJ/m2)"] = "Solar Radiation"
rename_dict["Rainfall(mm)"] = "Rainfall"
rename_dict["Snowfall (cm)"] = "Snowfall"
df.rename(columns=rename_dict, inplace=True)

In [3]:
# Predict Rented Bike Count
# Predictors Hour, Temp, Hum, Solar Rad, Rain, Snow, Seasons, Holiday
# Filter Functional Day (Times where bike sharing was unavailible)

# print(df.columns)
# print(df.describe())
# print(df["Functioning Day"].describe())
# print(df.shape)

# df_EDA = df.loc[df["Functioning Day"] != "Yes", :]
# print(df_EDA)

In [4]:
# First filter valid rows
df_filtered = df.loc[df["Functioning Day"] == "Yes", :]

X = df_filtered.iloc[:, 2:12]
y = df_filtered.loc[:, "Rented Bike Count"]

# Encode values in X
X = pd.get_dummies(X, drop_first=True)
X = pd.get_dummies(X, columns=["Hour"], drop_first=True)
print(X.columns)
# print(X.describe())

Index(['Temperature', 'Humidity', 'Wind speed', 'Visibility',
       'Dew point temperature', 'Solar Radiation', 'Rainfall', 'Snowfall',
       'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter', 'Hour_1',
       'Hour_2', 'Hour_3', 'Hour_4', 'Hour_5', 'Hour_6', 'Hour_7', 'Hour_8',
       'Hour_9', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14',
       'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_20',
       'Hour_21', 'Hour_22', 'Hour_23'],
      dtype='object')


In [5]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=22)

# validation split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=22)

std = StandardScaler()
ct = ColumnTransformer([
    ('std', StandardScaler(), list(rename_dict.values()))
], remainder='passthrough')
ct.fit_transform(X_train, y_test)
ct.transform(X_val)
ct.transform(X_test)
print(X_train.head())
print(X_train.describe())

# print(X_train.shape)
# print(y_train.shape)
# print(X_val.shape)
# print(y_val.shape)
# print(X_test.shape)
# print(y_test.shape)

      Temperature  Humidity  Wind speed  Visibility  Dew point temperature  \
1275         -8.6        68         3.3        2000                  -13.4   
5532         31.7        42         2.6        1868                   17.2   
5848         39.4        39         2.5        1897                   22.8   
5329         24.6        96         0.5         443                   23.9   
2157          2.4        95         2.5        1349                    1.6   

      Solar Radiation  Rainfall  Snowfall  Seasons_Spring  Seasons_Summer  \
1275             0.00       0.0       1.0               0               0   
5532             3.23       0.0       0.0               0               1   
5848             2.42       0.0       0.0               0               1   
5329             0.00       0.5       0.0               0               1   
2157             0.00       9.0       0.0               0               0   

      ...  Hour_14  Hour_15  Hour_16  Hour_17  Hour_18  Hour_19  Hou

In [6]:
# Create the model

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(6, input_shape=(X_train.shape[1],)))
    model.add(layers.Dropout(0.2, seed=22))
    model.add(layers.Dense(3, activation='relu'))
    model.add(layers.Dropout(0.2, seed=22))
    model.add(layers.Dense(1))
    model.summary()
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [ ]:
num_epochs = 20

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model = build_model()
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(
    X_test, y_test), batch_size=1, verbose=True, callbacks=[tensorboard_callback])
model.evaluate(X_test, y_test)

Metal device set to: Apple M1 Pro
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 210       
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dropout_1 (Dropout)         (None, 3)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 235
Trainable params: 235
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


2023-03-07 16:03:21.518272: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-07 16:03:21.518388: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-07 16:03:21.679976: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


   1/4536 [..............................] - ETA: 26:37 - loss: 104977.6641 - mae: 324.0026

2023-03-07 16:03:21.847302: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4528/4536 [============================>.] - ETA: 0s - loss: 346681.6250 - mae: 449.7550

2023-03-07 16:03:47.374724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4536/4536 [==============================] - 32s 7ms/step - loss: 346803.2188 - mae: 449.7125 - val_loss: 878607.4375 - val_mae: 722.1547
Epoch 2/20
4536/4536 [==============================] - 32s 7ms/step - loss: 281536.0312 - mae: 390.8530 - val_loss: 1000447.0625 - val_mae: 804.5308
Epoch 3/20
4536/4536 [==============================] - 32s 7ms/step - loss: 269141.2812 - mae: 379.3659 - val_loss: 941651.3125 - val_mae: 780.2028
Epoch 4/20
4536/4536 [==============================] - 32s 7ms/step - loss: 257217.8750 - mae: 370.7275 - val_loss: 833383.5000 - val_mae: 722.6204
Epoch 5/20
4536/4536 [==============================] - 32s 7ms/step - loss: 249745.4688 - mae: 365.3526 - val_loss: 965759.7500 - val_mae: 799.6802
Epoch 6/20
4536/4536 [==============================] - 31s 7ms/step - loss: 239834.3906 - mae: 356.2459 - val_loss: 1200916.0000 - val_mae: 918.3943
Epoch 7/20
4536/4536 [==============================] - 32s 7ms/step - loss: 229537.4688 - mae: 347.6832 - val_loss

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

plt.clf()   # clear figure